In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from env import host, user, password
import acquire
import wrangle
import prepare

import warnings
warnings.filterwarnings("ignore")

In [7]:
df = acquire.get_zillow_data()

df = wrangle.handle_nulls(df)

df.isnull().sum()

parcelid                        0
bathroomcnt                     0
bedroomcnt                      0
calculatedfinishedsquarefeet    0
fips                            0
latitude                        0
longitude                       0
lotsizesquarefeet               0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
landtaxvaluedollarcnt           0
taxamount                       0
logerror                        0
transactiondate                 0
dtype: int64

In [8]:
df = wrangle.prepare_zillow(df)

df = prepare.transaction_month(df)

df = prepare.tax_rate(df)

col_out = ["bathroomcnt", "bedroomcnt", "tax_rate", "calculatedfinishedsquarefeet", 
           "lotsizesquarefeet", "structuretaxvaluedollarcnt", "taxvaluedollarcnt", 
           "landtaxvaluedollarcnt"]

In [9]:
df = prepare.remove_outliers_iqr(df, col_out)

In [10]:
# Additional outlier removal
df = df[((df.bathroomcnt <= 7) & (df.bedroomcnt <= 7) & 
         (df.bathroomcnt > 0) & 
         (df.bedroomcnt > 0) & 
         (df.calculatedfinishedsquarefeet < 7000) & 
         (df.tax_rate < .05)
        )]

In [12]:
df = prepare.bed_bath_ratio(df)

In [13]:
df = prepare.better_names(df)

df.shape

(59930, 17)

In [14]:
# df.to_csv('zillow_df.csv')

In [15]:
df = pd.read_csv('zillow_df.csv')

In [16]:
df = df.drop(df.columns[0], axis=1)

In [17]:
df.head()

,longitude,latitude,age,month,bedroomcnt,bathroomcnt,square_footage,lot_size,house_value,land_value,full_value,tax_rate,Los_Angeles,Orange,Ventura,logerror,bed_bath_ratio
0,-117869207.0,33634931.0,19.0,1,4.0,3.5,3100.0,4506.0,485713.0,537569.0,1023282.0,0.010763,0,1,0,0.025595,1.142857
1,-119281531.0,34449266.0,50.0,1,2.0,1.0,1465.0,12647.0,88000.0,376000.0,464000.0,0.012225,0,0,1,0.055619,2.000000
2,-117823170.0,33886168.0,55.0,1,3.0,2.0,1243.0,8432.0,85289.0,479489.0,564778.0,0.011488,0,1,0,0.005383,1.500000
3,-118240722.0,34245180.0,47.0,1,4.0,3.0,2376.0,13038.0,108918.0,36225.0,145143.0,0.012247,1,0,0,-0.103410,1.333333
4,-118993991.0,34230044.0,35.0,1,3.0,2.0,1492.0,903.0,198640.0,132424.0,331064.0,0.010596,0,0,1,-0.020526,1.500000
